In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

2022-04-21 16:01:59.475066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-21 16:01:59.475108: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class modelNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(10))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [3]:
EPOCHS = 15
INIT_LR = 1e-3
BS = 64
CLASSES = 3

In [4]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [5]:
imagePaths = sorted(list(paths.list_images("augData/trialDataset")))
random.seed(42)
random.shuffle(imagePaths)

In [6]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    

In [7]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{'double_person': 947, 'single_person': 963, 'triple_person_3': 612}

In [8]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [9]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [10]:
print("[INFO] compiling model...")
model = modelNet.build(width=28, height=28, depth=3, classes=CLASSES)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


2022-04-21 16:02:20.394398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-21 16:02:20.394471: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-21 16:02:20.394508: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (awi-stavya): /proc/driver/nvidia/version does not exist
2022-04-21 16:02:20.394951: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/hom

In [11]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/15
29/29 [==============================] - 7s 174ms/step - loss: 0.9204 - accuracy: 0.5291
Epoch 2/15
29/29 [==============================] - 5s 179ms/step - loss: 0.7426 - accuracy: 0.6382
Epoch 3/15
29/29 [==============================] - 6s 191ms/step - loss: 0.6906 - accuracy: 0.6765
Epoch 4/15
29/29 [==============================] - 5s 182ms/step - loss: 0.6121 - accuracy: 0.7181
Epoch 5/15
29/29 [==============================] - 6s 199ms/step - loss: 0.5684 - accuracy: 0.7471
Epoch 6/15
29/29 [==============================] - 5s 185ms/step - loss: 0.5446 - accuracy: 0.7597
Epoch 7/15
29/29 [==============================] - 5s 179ms/step - loss: 0.5714 - accuracy: 0.7340
Epoch 8/15
29/29 [==============================] - 7s 236ms/step - loss: 0.5443 - accuracy: 0.7559
Epoch 9/15
29/29 [==============================] - 6s 204ms/step - loss: 0.4855 - accuracy: 0.7947
Epoch 10/15
29/29 [==============================] - 5s 187ms/step - loss

In [14]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_multi_class_3.png")

In [15]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(631, 3)


In [17]:
predictions

array([[4.1936138e-01, 5.7481551e-01, 5.8231219e-03],
       [3.5480526e-01, 6.2669963e-01, 1.8495129e-02],
       [1.5333109e-03, 1.6347321e-05, 9.9845028e-01],
       ...,
       [1.6576281e-01, 8.3356422e-01, 6.7292101e-04],
       [1.6733511e-01, 8.2773197e-01, 4.9329903e-03],
       [8.6295120e-02, 9.1102254e-01, 2.6823583e-03]], dtype=float32)

In [18]:
#importing the original dataset
data = []
labels = []
imagePaths = sorted(list(paths.list_images("Dataset")))

for imagePath in imagePaths:
    
    label = imagePath.split(os.path.sep)[-2]
    if label == "triple_person":
        label = 1
    else:
        label = 0
    labels.append(label)
    
    # for images
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)


In [19]:
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=lb.classes_))

                 precision    recall  f1-score   support

  double_person       0.81      0.54      0.65       251
  single_person       0.67      0.95      0.78       239
triple_person_3       0.96      0.82      0.89       141

       accuracy                           0.76       631
      macro avg       0.81      0.77      0.77       631
   weighted avg       0.79      0.76      0.75       631



### Trying out test set with 3 classification

In [20]:
test_imagepaths = sorted(list(paths.list_images("testSet")))
random.seed(42)
random.shuffle(test_imagepaths)

In [21]:
test_labels = []
test_data = []
for imagePath in test_imagepaths:    
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    test_data.append(image)

test_labels = np.array(test_labels)
test_data = np.array(test_data, dtype="float") / 255.0


In [24]:
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

In [25]:
test_predictions = model.predict(x=test_data, batch_size=32)

print(test_predictions.shape)

(221, 3)


In [26]:
test_predictions.argmax(axis=1)

array([0, 0, 2, 2, 0, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 2, 2, 0, 2, 1,
       0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 2, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 0, 0, 0, 1, 1, 2, 1, 0, 1,
       1, 0, 2, 0, 1, 2, 0, 0, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1, 2, 0, 1, 2,
       0, 1, 2, 2, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 2, 0, 1, 2, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 0, 0, 2, 0, 1, 2,
       0, 1, 0, 1, 1, 0, 0, 2, 1, 2, 1, 1, 0, 0, 2, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 2, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 1,
       0])

In [27]:
print(classification_report(test_labels.argmax(axis=1),test_predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.24      0.26      0.25        78
single_person       0.65      0.79      0.71        80
triple_person       0.29      0.19      0.23        63

     accuracy                           0.43       221
    macro avg       0.39      0.41      0.40       221
 weighted avg       0.40      0.43      0.41       221



### Code after this for predicting the dual class and is to be ignored

In [ ]:
data = np.array(data)
labels = np.array(labels)

labels = to_categorical(labels, num_classes=2)

In [ ]:
predictions = model.predict(x=data, batch_size=64)


In [ ]:
print(classification_report(labels.argmax(axis=1), predictions.argmax(axis=1),
                        target_names=["double_rider","triple_rider"]))

              precision    recall  f1-score   support

double_rider       0.91      0.52      0.67       948
triple_rider       0.03      0.22      0.05        63

    accuracy                           0.51      1011
   macro avg       0.47      0.37      0.36      1011
weighted avg       0.86      0.51      0.63      1011

